In [1]:
import sys
sys.path.append('../Data')

import pandas as pd
import numpy as np
import time
from CommentDataset import CommentDataset

comment_list_name = "20230627_mynumbercard_untenmenkyosyou"
comment_list_path = "2023_06_27/" + comment_list_name + ".csv"

# 指定すべき文字コードはファイルによる(cp932 or utf-8)
df = pd.read_csv("../Data/yahoo/"+comment_list_path, encoding="cp932", header=None)
# comment_list = df[0][1:].to_numpy().tolist()
comment_list = df[0][1:].to_numpy().tolist()
comment_dataset = CommentDataset(comment_list)
comment_dataset.delete_comment_specified_string(50)
comment_list = comment_dataset.comment_list
comment_dataset.formatted_input_hlda(appear_tagging_list=["名詞", "形容詞", "動詞"], stop_tagging_list=["助詞", "助動詞", "非自立"])

In [2]:
from hlda.sampler import HierarchicalLDA

#default
# n_samples = 500       # no of iterations for the sampler
n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 50   # the number of iterations between printing a brief summary of the topics so far
n_words = 10           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [3]:
hlda = HierarchicalLDA(comment_dataset.comment_list, comment_dataset.voc, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling

.................................................. 50
topic=0 level=0 (documents=530): 免許, れる, 運転, できる, 保険, 思う, 確認, 本人, カード, 情報, 
    topic=1 level=1 (documents=88): 国民, せる, 言う, 政治, これ, 一体, 政府, たち, 選挙, 政権, 
        topic=2 level=2 (documents=3): 与党, 潰す, 示す, 公明党, 付ける, 次回, 制度, 老人, 意外, 横暴, 
        topic=10 level=2 (documents=10): 視力, 検査, 皆さん, 不信, ダメージ, モデル, 気楽, 公共, 面々, デカ, 
        topic=52 level=2 (documents=5): 毎日, 軽視, しな, 家庭, 名誉, 酷使, 狙い, 成長, 結局, 恥ずかしい, 
        topic=58 level=2 (documents=15): 機能, コピー, 計画, 破綻, 将来, 実現, 等々, 読む, 助かる, 対面, 
        topic=59 level=2 (documents=9): 人間, メリット, 悪い, オレ, 結局, 認識, 苦笑, 関わる, デメリット, 保管, 
        topic=64 level=2 (documents=9): 無くす, 保持, 連携, 使い, 押し付ける, 一体化, 期間, 離す, 肌身, 省庁, 
        topic=71 level=2 (documents=7): 例える, 内閣, 終わる, 出す, 構成, 土地, 納得, 傲慢, 金庫, 金持ち, 
        topic=72 level=2 (documents=8): 企業, アメリカ, 家畜, 上手い, 独裁, 壊す, 子供, 国家, 領域, 暴走, 
        topic=78 level=2 (documents=5): 無視, ヶ月, 日本国, 江戸, 民意, 時代, 刻む, ぶる, 開発, 自身, 
        

In [4]:
from model.HldaModel import ExpandHldaModel
import os

create_date = "2023_08_01"
parameter_name = "default"
model_name = comment_list_name + "_" + parameter_name + ".pickle"
model_dir = "model/pickle/" + create_date + "/" +model_name
create_date_path = "model/pickle/" + create_date

if not os.path.exists(create_date_path):
    os.makedirs(create_date_path)

expandHlda = ExpandHldaModel(pickle_path=hlda)
expandHlda.save_zipped_pickle(filename=model_dir)

In [5]:
# expandHlda.get_topic_multi_document(comment_list=comment_list, corpus=comment_dataset.comment_list)

In [6]:
expandHlda.print_phrases(comment_list=comment_list, corpus=comment_dataset.comment_list, n_phrase=3, with_score=True)

topic=0 level=0 (documents=530): 免許 (0.079),運転免許 (0.061),免許確認 (0.053),
    topic=1 level=1 (documents=90): 国民 (0.048),政府 (0.047),日本政府 (0.039),
        topic=58 level=2 (documents=13): 指紋認証 (0.058),コピー (0.057),指紋 (0.041),
        topic=145 level=2 (documents=7): 中国 (0.104),一族 (0.056),職員 (0.026),
スマートフォン (0.058),7 level=2 (documents=11): スマートフォン (0.087),スマートフォン料金 (0.064),
        topic=628 level=2 (documents=10): エラー (0.054),それぞれ (0.050),警官 (0.027),
        topic=678 level=2 (documents=2): 余計な税金 (0.085),多額 (0.022),まま (0.014),
        topic=693 level=2 (documents=3): 能力 (0.044),バックアップ (0.043),ハッキング (0.043),
        topic=729 level=2 (documents=6): 無能 (0.064),無能政府 (0.039),危険 (0.034),
        topic=760 level=2 (documents=4): オランダ (0.072),オランダ政府 (0.040),外国 (0.038),
        topic=776 level=2 (documents=9): 一つ (0.041),警察官 (0.035),どれ (0.026),
        topic=781 level=2 (documents=5): インフラ整備 (0.082),義務 (0.041),法案成立 (0.039),
        topic=783 level=2 (documents=7): 口座 (0.036),未来 (0.032),証券口座 (0.02

In [7]:
# expandHlda.print_multi_node(comment_list=comment_list, corpus=comment_dataset.comment_list, n_phrase=3, with_score=True)

In [8]:
expandHlda.get_topic_by_sentence(comment_list=comment_list, corpus=comment_dataset.comment_list, n_words=5)

0:今の日本の運転免許証ってとても優秀なものだと思うんですけどね。
node_id:628, node_words:エラー(4.0),それぞれ(4.0),推し進める(3.0),出る(3.0),脆弱(3.0), probability:0.025
1:数年に一度必ず本人出向いて更新するから身元確認にもなるし顔写真ついてるからすぐその場で本人確認できるし。
node_id:116, node_words:更新(27.0),ゴールド(6.0),ブルー(4.0),即日(3.0),会場(2.0), probability:0.189
2:なんにせよあらゆることを一つだけに絞るとそれがダメになった時にどうしようもなくなるのでリスクが大きくなると思います。
node_id:776, node_words:ダメ(6.0),くっつける(2.0),警察官(2.0),絡む(2.0),一つ(2.0), probability:0.17
3:昭和おっさんの記憶ですが、テレビとビデオをくっつけたものは結局すたれたし、ファミコンとディスクシステム？だったかな？、をくっつけてもダメでしたね。
node_id:776, node_words:ダメ(6.0),くっつける(2.0),警察官(2.0),絡む(2.0),一つ(2.0), probability:0.189
4:一つダメになると他も使えなくなるので。
node_id:776, node_words:ダメ(6.0),くっつける(2.0),警察官(2.0),絡む(2.0),一つ(2.0), probability:0.151
------------------------------------------------------------------
0:紛失時のリスクをマイナカードに集約してしまうと、再発行されるまで病院にも行けない、車にも乗れない、となりますね。
node_id:9, node_words:紛失(31.0),発行(30.0),リスク(14.0),無くす(14.0),病院(13.0), probability:0.266
1:役所が休みの土日や盆暮れだと、相当生活に影響が出そう。
node_id:791, node_words:普及(7.0),値する(2.0),普通(2.0),身元(1.0),今更(1.0), probab

In [ ]:
# sentence="河野、岸田お前たちの頭狂っているのではないのか、これ以上馬鹿な政策は止めろ、狂気の沙汰としか見えないぞ"
# expandHlda.get_topic_one_sentence(sentence=sentence, corpus=comment_dataset.comment_list, n_words=10, node_id=7)

In [ ]:
# sentence="河野、岸田お前たちの頭狂っているのではないのか、これ以上馬鹿な政策は止めろ、狂気の沙汰としか見えないぞ"
# expandHlda.get_topic_one_sentence_with_color(sentence=sentence, corpus=comment_dataset.comment_list, n_words=10, node_id=7)

In [ ]:
expandHlda.get_topic_with_color(comment_list=comment_list, corpus=comment_dataset.comment_list, n_words=10)

0:今の日本の運転免許証ってとても優秀だものだと思うんですけどね。
node_id:533, node_words:デジタル(12.0),対応(6.0),世界(5.0),企業(4.0),状態(4.0),日本(4.0),必須(3.0),優先(3.0),言える(3.0),大事(3.0), probability:0.033
1:数年に一度必ず本人出向くて更新するから身元確認にもなるき顔写真ついてるからすぐその場で本人確認できるし。
node_id:764, node_words:更新(24.0),ゴールド(4.0),即日(3.0),読み取り(2.0),検査(2.0),いく(2.0),こっち(2.0),手間(2.0),装置(2.0),政権(2.0), probability:0.255
2:**なんにするあらゆることを一つだけに絞るとそれがダメになるた時にどうしようもなくなるのでリスクが大きいなると思うます。
node_id:854, node_words:最初(3.0),一つ(3.0),ダメ(3.0),まとめる(3.0),振り回す(3.0),くっつける(2.0),活用(2.0),様々(2.0),根源(2.0),ツール(2.0), probability:0.092
3:*
node_id:854, node_words:最初(3.0),一つ(3.0),ダメ(3.0),まとめる(3.0),振り回す(3.0),くっつける(2.0),活用(2.0),様々(2.0),根源(2.0),ツール(2.0), probability:0.0
4:昭和おっさんの記憶ですが、テレビとビデオをくっつけるたものは結局すたれるたし、ファミコンとディスクシステム？だたかな？、をくっつけるてもダメですたね。
node_id:854, node_words:最初(3.0),一つ(3.0),ダメ(3.0),まとめる(3.0),振り回す(3.0),くっつける(2.0),活用(2.0),様々(2.0),根源(2.0),ツール(2.0), probability:0.092
5:一つダメになると他も使えるないなるので。
node_id:854, node_words:最初(3.0),一つ(3.0),ダメ(3.0),まとめる(3.0),振り回す(3.0),くっつける(2.0),活用(2.0),様々(2.0),根

In [9]:
expandHlda.print_topic_multi_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=708)

No.1
「国民の非難轟轟のマイナ保険証に加えて運転免許証もマイナンバーに一本化するなんてとんでもない話しです。個人情報を集約したマイナンバーを常に形態しなければならない事は国民にとってリスクしかありません。そもそも欧州をはじめとする諸外国はマイナンバーを制度化して失敗し結局、廃止にシテイマス。それなのに何故、政府自民は強行するのか。そんなにマイナ制度をやりたければ、政府自民議員だけで導入して、社会実験として資産や収入、口座、保険証、運転免許証などあらゆる情報を一本化して国民に不正なカネの動きがないか目が届く制度にすれば良いと思います。」
0.927
No.2
「運転免許や保険証とかの個人の情報をマイナンバーに集約するというのはそれはそれでわからんことはないんだけども、それを物理的なカードに集約してしまうってのは、紛失の際のリスクや暗証番号を間違えたときの再設定とかを考慮すると、むりやり進める必要はないけども、どうして突っ走るのかね、自民党は本当のあほうなのか？、ファザコン河野も民主のせいというならさっさとやめれ」
0.853
No.3
「保険証や免許証をマイナンバーカードに集約したい人もいるし、集約したくない人もいる。だから、選択肢を増やして、それぞれの移行状況を見ながら、その割合で決めればいいのに、一気にやってしまおうと、選択の自由を強引に奪おうとするから反発される。自由民主党じゃなく、不自由非民主党じゃんか。」
0.847
No.4
「オランダに駐在した時、オランダの運転免許証に書き換えした際に日本の運転免許証を取り上げられました。日本の運転免許証と交換したことになります。在オランダ日本大使館とオランダ政府との間で特別に日本の運転免許証を本人に返す約束ができていたので、数カ月後に戻ってきました。もし、マイナンバーカードが運転免許証を兼ねるとなれば、免許の記載事項が明記されていないので外国で用を足さない。記載されたとして、取り上げられて返却されないとなった場合、帰国した時、どうすればいいのでしょうかね？　外国の運転免許証でマイナンバーカードを発行してくれるのでしょうか？　今の政府はこのようなことまで考えていないでしょうね。」
0.729
No.5
「あらゆる情報がマイナンバーカードに集約されて、そのうち犬猫みたいにマイクロチップが埋め込まれるようになるのでしょう